## Kautilya Assignment 3

In [1]:
!pip install selenium

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import selenium
import re
import time

In [3]:
from selenium.webdriver import Chrome, Firefox
from selenium.webdriver.support.ui import Select
from selenium import webdriver

In [4]:
# subject to change to your own driver
# browser = Firefox()
browser = webdriver.Firefox()

In [5]:
url='https://appsrv.pace.edu/ScheduleExplorerLive/'
browser.get(url)

## Interaction 

In [6]:
term=Select(browser.find_element_by_id('checkterm'))
term.select_by_visible_text('Fall 2020')
time.sleep(1)

In [7]:
level=Select(browser.find_element_by_id('level'))
level.select_by_visible_text('Graduate')

time.sleep(1)

In [8]:
subject=Select(browser.find_element_by_id('subject'))
subjects= [subject.text for subject in subject.options]
time.sleep(1)

In [9]:
IS_text=[subject for subject in subjects if 'IS' in subject][0]
subject.select_by_visible_text(IS_text)
search_button=browser.find_element_by_id('submitbutton')
search_button.click()

In [10]:
time.sleep(3)

## Increase the table
dd_number=Select(browser.find_element_by_name('mainResultsTable_length'))
dd_number.select_by_value('100')

### 1. Extract the list from the page

In [11]:
html=browser.page_source
courses=pd.read_html(html)[0]
soup=BeautifulSoup(html,'html.parser')
links=soup.find_all('a',{'href':True})

## I will only get the links for CRN numbers

links_selected=soup.find_all('a',{'data-toggle':'modal'})
crn_links=[link for link in links_selected  if link.next.strip().isnumeric()]

### 2. Browse the links 

<b style='color:red' >Problem: </b> Links do not have <mark>"href"</mark> arguments

We will need to simulate clicking on these links to trigger the java script functions 

Let's write a function that will return one pop up message

In [12]:
def read_crn(crn_link):
    link=browser.find_element_by_partial_link_text(crn_link.text.strip())
    time.sleep(1)
    link.click()
    popup_content=browser.find_element_by_id('modalBodyContentDiv')
    
    df=pd.read_html(popup_content.get_attribute('innerHTML'))[0].T
    df=pd.read_html(popup_content.get_attribute('innerHTML'))[0].T
    header=df.iloc[0]
    df=df[1:]
    df.columns=header
   
    pop_up_header=browser.find_element_by_class_name('modal-header')
    header_close=pop_up_header.find_element_by_tag_name('Button')
    time.sleep(2)
    header_close.click()
    return df

In [13]:
dfs=[]
for crn_link in crn_links:
    try:
        df=read_crn(crn_link)
        dfs.append(df)
    except Exception:
        continue

In [14]:
all_is_cours_decs=pd.concat(dfs)

In [15]:
all_is_cours_decs.head()

,COURSE TITLE:,SUBJECT/COURSE#:,CRN:,CREDIT RANGE:,SCHOOL:,DEPARTMENT:,LEVEL:,COREQUISITE:,PREREQUISITE:,DESCRIPTION:
1,Advanced Spreadsheet Skills,IS 522,73592,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: This is an online course t...
1,Introduction to Coding,IS 612,70426,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course provides an in...
1,Database Management Systems,IS 613,70376,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
1,Database Management Systems,IS 613,71721,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
1,Database Management Systems,IS 613,72779,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...


## Combine datasets 

In [16]:
all_is_cours_decs['CRN:']=pd.to_numeric(all_is_cours_decs['CRN:'])

In [17]:
main_table=pd.read_html(html)[0]

In [18]:
main_table.head()

,CRN,Subject,Course#,Title,Sched Type,Credits,Campus,Instructional Method*,Section Comments,Days**,Time,Capacity,Seats Avail.,Instructor,More Info
0,73592,IS,522,Advanced Spreadsheet Skills,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,NaN,25,18,Henry Gaylord,More Info
1,70426,IS,612,Introduction to Coding,LEC,3,NYC,Web-assisted (WA),NaN,W,06:10pm-09:00pm,23,03,Dhruvil Gandhi,More Info
2,70376,IS,613,Database Management Systems,LEC,3,NYC,Web-assisted (WA),NaN,T,06:10pm-09:00pm,30,CLOSED,Nathifa Lewis,Prerequisites
3,71721,IS,613,Database Management Systems,LEC,3,Online,Synchronous (SYNC),NaN,T,06:10pm-09:00pm,30,01,Helen Uzamere,Prerequisites
4,72779,IS,613,Database Management Systems,LEC,3,Online,Mixed (MIXAS),NaN,S,09:00am-12:00pm,30,14,Paul Dantzig,Prerequisites


In [19]:
all_is_cours_decs=all_is_cours_decs.rename({'CRN:':'CRN'}, axis='columns')
all_is_cours_decs.head()

,COURSE TITLE:,SUBJECT/COURSE#:,CRN,CREDIT RANGE:,SCHOOL:,DEPARTMENT:,LEVEL:,COREQUISITE:,PREREQUISITE:,DESCRIPTION:
1,Advanced Spreadsheet Skills,IS 522,73592,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: This is an online course t...
1,Introduction to Coding,IS 612,70426,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course provides an in...
1,Database Management Systems,IS 613,70376,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
1,Database Management Systems,IS 613,71721,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
1,Database Management Systems,IS 613,72779,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...


In [20]:
combined_IS=main_table.merge(all_is_cours_decs, how='left', on='CRN')

In [21]:
combined_IS.columns

Index(['CRN', 'Subject', 'Course#', 'Title', 'Sched Type', 'Credits', 'Campus',
       'Instructional Method*', 'Section Comments', 'Days**', 'Time',
       'Capacity', 'Seats Avail.', 'Instructor', 'More Info', 'COURSE TITLE:',
       'SUBJECT/COURSE#:', 'CREDIT RANGE:', 'SCHOOL:', 'DEPARTMENT:', 'LEVEL:',
       'COREQUISITE:', 'PREREQUISITE:', 'DESCRIPTION:'],
      dtype='object')

In [22]:
combined_IS

,CRN,Subject,Course#,Title,Sched Type,Credits,Campus,Instructional Method*,Section Comments,Days**,...,More Info,COURSE TITLE:,SUBJECT/COURSE#:,CREDIT RANGE:,SCHOOL:,DEPARTMENT:,LEVEL:,COREQUISITE:,PREREQUISITE:,DESCRIPTION:
0,73592,IS,522,Advanced Spreadsheet Skills,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,More Info,Advanced Spreadsheet Skills,IS 522,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: This is an online course t...
1,70426,IS,612,Introduction to Coding,LEC,3,NYC,Web-assisted (WA),NaN,W,...,More Info,Introduction to Coding,IS 612,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course provides an in...
2,70376,IS,613,Database Management Systems,LEC,3,NYC,Web-assisted (WA),NaN,T,...,Prerequisites,Database Management Systems,IS 613,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
3,71721,IS,613,Database Management Systems,LEC,3,Online,Synchronous (SYNC),NaN,T,...,Prerequisites,Database Management Systems,IS 613,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
4,72779,IS,613,Database Management Systems,LEC,3,Online,Mixed (MIXAS),NaN,S,...,Prerequisites,Database Management Systems,IS 613,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
5,72779,IS,613,Database Management Systems,LEC,3,Online,Mixed (MIXAS),NaN,S,...,Prerequisites,Database Management Systems,IS 613,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
6,70063,IS,617,Information Systems Principles,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,More Info,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,71371,IS,620,IS and Organizational Strategy,LEC,3,NYC,HyFlex (HYFLX),NaN,W,...,More Info,Information Systems and Organizational Strategy,IS 620,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course brings to life...
8,70377,IS,623,Info Sys Design & Devlpmt,LEC,3,NYC,HyFlex (HYFLX),NaN,M,...,More Info,Information Systems Design and Development,IS 623,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Not open to students who have taken IS 621. Co...
9,71427,IS,623,Info Sytems Design & Devlpmt,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,More Info,Information Systems Design and Development,IS 623,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Not open to students who have taken IS 621. Co...


In [25]:
def clean_description(text,clean_word='Course Description:'):
    if isinstance(text,str) and text.startswith(clean_word):
        index = text.index(clean_word) + len(clean_word)
        return text[index:]
    else:
        return text

In [26]:
 combined_IS['DESCRIPTION:'] = combined_IS['DESCRIPTION:'].apply(clean_description)

In [37]:
import nltk
nltk.download("stopwords")
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kautilya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kautilya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:
stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()

def clean(text):
    text=re.sub(r'[^\w\s]',' ',text).lower()   ### Remove punctuation by removing non-space alpha-num characters
    words=re.split(r'\W+', text)                   ### Split by remaining non-alphanumeric characters 
    non_stop_words=[word for word in words if word not in stopword]        ### Remove stopwords
    lem_words= [wn.lemmatize(word) for word in non_stop_words]             ### Clean text ==
    clean_text= ' '.join(lem_words)
    return clean_text

In [39]:
combined_IS.columns

Index(['CRN', 'Subject', 'Course#', 'Title', 'Sched Type', 'Credits', 'Campus',
       'Instructional Method*', 'Section Comments', 'Days**', 'Time',
       'Capacity', 'Seats Avail.', 'Instructor', 'More Info', 'COURSE TITLE:',
       'SUBJECT/COURSE#:', 'CREDIT RANGE:', 'SCHOOL:', 'DEPARTMENT:', 'LEVEL:',
       'COREQUISITE:', 'PREREQUISITE:', 'DESCRIPTION:'],
      dtype='object')

In [40]:
df_IS = combined_IS[['CRN', 'Subject', 'Course#', 'Title','DESCRIPTION:']]

In [41]:
df_IS = df_IS.dropna()
df_IS

,CRN,Subject,Course#,Title,DESCRIPTION:
0,73592,IS,522,Advanced Spreadsheet Skills,This is an online course that introduces stud...
1,70426,IS,612,Introduction to Coding,This course provides an introduction to progr...
2,70376,IS,613,Database Management Systems,This course focuses on the theoretical and pr...
3,71721,IS,613,Database Management Systems,This course focuses on the theoretical and pr...
4,72779,IS,613,Database Management Systems,This course focuses on the theoretical and pr...
5,72779,IS,613,Database Management Systems,This course focuses on the theoretical and pr...
7,71371,IS,620,IS and Organizational Strategy,This course brings to life the latest busines...
8,70377,IS,623,Info Sys Design & Devlpmt,Not open to students who have taken IS 621. Co...
9,71427,IS,623,Info Sytems Design & Devlpmt,Not open to students who have taken IS 621. Co...
10,72669,IS,628,Research Mthds for User Exp.,"In User Experience, the foundation is researc..."


In [42]:
df_IS['clean_text'] = df_IS['DESCRIPTION:'].apply(clean)

In [43]:
df_IS

,CRN,Subject,Course#,Title,DESCRIPTION:,clean_text
0,73592,IS,522,Advanced Spreadsheet Skills,This is an online course that introduces stud...,online course introduces student feature micr...
1,70426,IS,612,Introduction to Coding,This course provides an introduction to progr...,course provides introduction programming pyth...
2,70376,IS,613,Database Management Systems,This course focuses on the theoretical and pr...,course focus theoretical practical aspect fil...
3,71721,IS,613,Database Management Systems,This course focuses on the theoretical and pr...,course focus theoretical practical aspect fil...
4,72779,IS,613,Database Management Systems,This course focuses on the theoretical and pr...,course focus theoretical practical aspect fil...
5,72779,IS,613,Database Management Systems,This course focuses on the theoretical and pr...,course focus theoretical practical aspect fil...
7,71371,IS,620,IS and Organizational Strategy,This course brings to life the latest busines...,course brings life latest business research c...
8,70377,IS,623,Info Sys Design & Devlpmt,Not open to students who have taken IS 621. Co...,open student taken 621 course description cour...
9,71427,IS,623,Info Sytems Design & Devlpmt,Not open to students who have taken IS 621. Co...,open student taken 621 course description cour...
10,72669,IS,628,Research Mthds for User Exp.,"In User Experience, the foundation is researc...",user experience foundation research required ...


In [44]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer()

In [45]:
X_counts = count_vect.fit_transform(df_IS['clean_text'])
vocab=count_vect.get_feature_names()

In [46]:
vs_model=pd.DataFrame(X_counts.toarray(), columns=vocab)
vs_model

,2016,24,27000,27001,27002,600,621,695q,access,accessing,...,wide,widening,within,without,work,workbook,worksheet,world,written,yet
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


### Top 20 words

In [47]:
top_words = vs_model.sum().sort_values(ascending=False)
top_words.head(20)

data           68
course         48
management     45
system         41
database       26
security       25
information    23
student        21
project        18
include        17
topic          17
development    15
design         15
software       13
structure      13
business       12
research       12
application    12
rotation       11
plv            10
dtype: int64

### Longest Description

In [68]:
top_len = df_IS['DESCRIPTION:'].agg(len).sort_values(ascending=False)

In [94]:
# top_len.idxmax()
top_len

28    1758
34    1243
19    1213
32     765
7      728
0      667
13     660
15     660
14     660
10     576
26     554
9      498
8      498
5      494
4      494
3      494
2      494
17     453
18     453
22     396
1      396
21     396
16     383
24     311
12     271
11     271
Name: DESCRIPTION:, dtype: int64

In [99]:
top_index = top_len.idxmax()
top_index

28

In [100]:
df_IS

,CRN,Subject,Course#,Title,DESCRIPTION:,clean_text
0,73592,IS,522,Advanced Spreadsheet Skills,This is an online course that introduces stud...,online course introduces student feature micr...
1,70426,IS,612,Introduction to Coding,This course provides an introduction to progr...,course provides introduction programming pyth...
2,70376,IS,613,Database Management Systems,This course focuses on the theoretical and pr...,course focus theoretical practical aspect fil...
3,71721,IS,613,Database Management Systems,This course focuses on the theoretical and pr...,course focus theoretical practical aspect fil...
4,72779,IS,613,Database Management Systems,This course focuses on the theoretical and pr...,course focus theoretical practical aspect fil...
5,72779,IS,613,Database Management Systems,This course focuses on the theoretical and pr...,course focus theoretical practical aspect fil...
7,71371,IS,620,IS and Organizational Strategy,This course brings to life the latest busines...,course brings life latest business research c...
8,70377,IS,623,Info Sys Design & Devlpmt,Not open to students who have taken IS 621. Co...,open student taken 621 course description cour...
9,71427,IS,623,Info Sytems Design & Devlpmt,Not open to students who have taken IS 621. Co...,open student taken 621 course description cour...
10,72669,IS,628,Research Mthds for User Exp.,"In User Experience, the foundation is researc...",user experience foundation research required ...


In [101]:
df_IS_index = df_IS.index
df_IS_index

Int64Index([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
            18, 19, 21, 22, 24, 26, 28, 32, 34],
           dtype='int64')

In [102]:
highest_word_course_index = df_IS_index.get_loc(top_index)
highest_word_course_index

23

In [103]:
df_IS.iloc[highest_word_course_index]

CRN                                                         72599
Subject                                                        IS
Course#                                                       669
Title                              Big Data & Information Systems
DESCRIPTION:     Data and analytics are changing the world and...
clean_text       data analytics changing world way making deci...
Name: 28, dtype: object

### Number of words

In [105]:
n_words = vs_model.iloc[highest_word_course_index].sum()
n_words

160